In [36]:
from datetime import datetime
schema = [{'name': 'id', "type":"id" }, {'name':'feature1', 'type':'float'}, {'name':'feature2', 'type':'string'},{'name':'feature3', 'type':'datetime'},{'name':'feature4', 'type':'bool'}]
features = [[1,2,3,4], [10.1, 11.1, 12.1, 13.1], ['hello', 'hi','bye','dd'],[datetime(2023,6,10),datetime(2023,6,10),datetime(2023,6,10),datetime(2023,6,10)],[True, False, True, False]]
tags = [[1, 0], [2, 0],[3,1], [4, 1]]
date = [datetime(2023,6,10),datetime(2023,6,10),datetime(2023,6,10),datetime(2023,6,10)]
bo = [True, False, True, False]
print([int(x) for x in bo])
print(date)
print([int(x.strftime("%Y%m%d%H%M%S")) for x in date])


[1, 0, 1, 0]
[datetime.datetime(2023, 6, 10, 0, 0), datetime.datetime(2023, 6, 10, 0, 0), datetime.datetime(2023, 6, 10, 0, 0), datetime.datetime(2023, 6, 10, 0, 0)]
[20230610000000, 20230610000000, 20230610000000, 20230610000000]


In [37]:
import math
def one_hot_encode(arr):
    unique = list(set(arr))
    print("unique", unique)
    mapping = {}
    for i, x in enumerate(unique):
        mapping[x] = i
    print("mapping", mapping)
    one_hot_encoder = []
    for a in arr:
        res = [0]*len(arr)
        res[mapping[a]] = 1
        one_hot_encoder.append(res)
    return one_hot_encoder
print(one_hot_encode(['hello', 'hi','bye','dd']))

def get_std_scaler(arr):
    arr_mean = sum(arr)/len(arr)
    var = sum(pow(x-arr_mean,2) for x in arr)/len(arr)
    std = math.sqrt(var)
    return [(x-arr_mean)/std for x in arr]

def get_minmax_scaler(arr):
    arr_min, arr_max = min(arr), max(arr)
    sub = arr_max-arr_min
    return [(x-arr_min)/sub for x in arr]

arr = [1,2,4,5,6,7,7]
print(get_std_scaler(arr))
print(get_minmax_scaler(arr))

def preprossing(schema, features, tags):
    scaledX = []
    lables = {}
    for id, l in tags:
        lables[id] = l
    y = [lables[x] for x in features[0]]
    for i in range(1, len(schema)):
        if schema[i]['type'] == 'float':
            scaledX.append(get_minmax_scaler(features[i]))
        elif schema[i]['type'] == 'string':
            encoders = one_hot_encode(features[i])
            for e in encoders:
                scaledX.append(e)
        elif schema[i]['type'] == 'datetime':
            scaledX.append([int(x.strftime("%Y%m%d%H%M%S")) for x in features[i]])
        elif schema[i]['type'] == 'bool':
            scaledX.append([int(x) for x in features[i]])
    print("scaleX is: ", scaledX)
    
    
    return list(map(list, zip(*scaledX))), y
print(preprossing(schema, features, tags))


    

unique ['hi', 'hello', 'dd', 'bye']
mapping {'hi': 0, 'hello': 1, 'dd': 2, 'bye': 3}
[[0, 1, 0, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0]]
[-1.627361387260298, -1.1717001988274145, -0.2603778219616476, 0.1952833664712359, 0.6509445549041194, 1.106605743337003, 1.106605743337003]
[0.0, 0.16666666666666666, 0.5, 0.6666666666666666, 0.8333333333333334, 1.0, 1.0]
unique ['hi', 'hello', 'dd', 'bye']
mapping {'hi': 0, 'hello': 1, 'dd': 2, 'bye': 3}
scaleX is:  [[0.0, 0.3333333333333333, 0.6666666666666666, 1.0], [0, 1, 0, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0], [20230610000000, 20230610000000, 20230610000000, 20230610000000], [1, 0, 1, 0]]
([[0.0, 0, 1, 0, 0, 20230610000000, 1], [0.3333333333333333, 1, 0, 0, 0, 20230610000000, 0], [0.6666666666666666, 0, 0, 0, 1, 20230610000000, 1], [1.0, 0, 0, 1, 0, 20230610000000, 0]], [0, 0, 1, 1])


In [4]:
#%pip install scikit-learn
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

data = {'Employee id': [10, 20, 15, 25, 30],
        'Gender': ['M', 'F', 'F', 'M', 'F'],
        'Remarks': ['Good', 'Nice', 'Good', 'Great', 'Nice'],
        }

df = pd.DataFrame(data)
categorical_columns = df.select_dtypes(include=['object']).columns.tolist()

encoder = OneHotEncoder(sparse_output=False)

one_hot_encoded = encoder.fit_transform(df[categorical_columns])

one_hot_df = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out(categorical_columns))

df_encoded = pd.concat([df, one_hot_df], axis=1)

df_encoded = df_encoded.drop(categorical_columns, axis=1)

print(f"Encoded Employee data : \n{df_encoded}")

Encoded Employee data : 
   Employee id  Gender_F  Gender_M  Remarks_Good  Remarks_Great  Remarks_Nice
0           10       0.0       1.0           1.0            0.0           0.0
1           20       1.0       0.0           0.0            0.0           1.0
2           15       1.0       0.0           1.0            0.0           0.0
3           25       0.0       1.0           0.0            1.0           0.0
4           30       1.0       0.0           0.0            0.0           1.0


In [6]:
#implement one hot encoding without calling sklearn
import numpy as np

colors = ["red", "green", "yellow", "red", "blue"]
total_colors = ["red", "green", "blue", "black", "yellow"]
mapping = {}
for x in range(len(total_colors)):
    mapping[total_colors[x]] = x
one_hot_encoder= []
#print(mapping)
for c in colors:
    arr = list(np.zeros(len(total_colors), dtype=int))
    arr[mapping[c]] = 1
    one_hot_encoder.append(arr)
print(one_hot_encoder)

{'red': 0, 'green': 1, 'blue': 2, 'black': 3, 'yellow': 4}
[[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 0, 1], [1, 0, 0, 0, 0], [0, 0, 1, 0, 0]]


In [3]:

import datetime

def one_hot_encode(strings):
    # Identify all unique string values
    unique_strings = list(set(strings))
    unique_strings.sort()  # Sorting to ensure consistent ordering

    # Create a dictionary to map each unique string to an index
    string_to_index = {string: index for index, string in enumerate(unique_strings)}

    # Create the one-hot encoded vectors
    one_hot_vectors = []
    for string in strings:
        vector = [0] * len(unique_strings)
        index = string_to_index[string]
        vector[index] = 1
        one_hot_vectors.append(vector)

    return one_hot_vectors

def prepare_data_for_logistic_regression(schema, training_data, label_data):
    # Helper functions for type conversion
    def convert_value(value, type_name):
        if type_name == 'int':
            return int(value)
        elif type_name == 'float':
            return float(value)
        elif type_name == 'bool':
            return 1 if value else 0
        elif type_name == 'str':
            return value  # We'll handle string conversion separately
        elif type_name == 'date':
            return (datetime.datetime.strptime(value, "%Y-%m-%d") - datetime.datetime(1970, 1, 1)).days
        elif type_name == 'datetime':
            return (datetime.datetime.strptime(value, "%Y-%m-%d %H:%M:%S") - datetime.datetime(1970, 1, 1)).total_seconds()
        else:
            raise ValueError(f"Unsupported type: {type_name}")

    # Create a list of column names and types
    column_names = [col['name'] for col in schema]
    column_types = {col['name']: col['type'] for col in schema}

    # Collect all string columns for one-hot encoding
    string_columns = [col['name'] for col in schema if col['type'] == 'str']
    string_values = {col: [] for col in string_columns}

    # Gather string values for each string column
    for record in training_data:
        for col in string_columns:
            idx = column_names.index(col)
            string_values[col].append(record[idx])

    # Perform one-hot encoding for all string columns
    one_hot_encoded = {col: one_hot_encode(string_values[col]) for col in string_columns}

    # Prepare the feature vectors
    feature_vectors = []
    for record in training_data:
        feature_vector = []
        for i, value in enumerate(record):
            column_name = column_names[i]
            column_type = column_types[column_name]
            if column_type != 'str':
                feature_vector.append(convert_value(value, column_type))
            else:
                # Append one-hot encoded vector for this string value
                feature_vector.extend(one_hot_encoded[column_name].pop(0))
        
        # Append the label at the end of the feature vector
        record_id = record[0]
        label = label_data.get(record_id, 0)
        feature_vector.append(label)
        feature_vectors.append(feature_vector)

    return feature_vectors

# Example usage:
schema = [
    {'type': 'int', 'name': 'id'},
    {'type': 'int', 'name': 'feature1'},
    {'type': 'float', 'name': 'feature2'},
    {'type': 'str', 'name': 'feature3'},
    {'type': 'date', 'name': 'feature4'}
]

training_data = [
    [1, 10, 0.5, 'A', '2022-01-01'],
    [2, 20, 1.5, 'B', '2023-01-01']
]

label_data = {
    1: 1,
    2: 0
}

feature_vectors = prepare_data_for_logistic_regression(schema, training_data, label_data)
print(feature_vectors)

[[1, 10, 0.5, 1, 0, 18993, 1], [2, 20, 1.5, 0, 1, 19358, 0]]


In [1]:
import string
from collections import defaultdict
def solution(words):
    
    counters = {x: defaultdict(int) for x in list(string.ascii_lowercase)}
    maximum = {x: 0 for x in list(string.ascii_lowercase)}
    
    for word in words:
        chars = set(word)
        for c1 in chars:
            for c2 in chars:
                if c1 != c2:
                    counters[c1][c2]+=1
                    maximum[c1] = max(maximum[c1], counters[c1][c2])
    res = defaultdict(list)
    for c in maximum.keys():
        for letter, count in counters[c].items():
            if count == maximum[c]:
                res[c].append(letter)
    return res

print(solution(['abc', 'bcd', 'cde']))

defaultdict(<class 'list'>, {'a': ['b', 'c'], 'b': ['c'], 'c': ['b', 'd'], 'd': ['c'], 'e': ['d', 'c']})
